In [1]:
import pandas as pd
import numpy as np
import lib as l
import re
import string
import math
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter("ignore")
#from fancyimpute import KNN
from datetime import timedelta
from sklearn.cluster import KMeans
#from pyod.models.iforest import IForest
#from pyod.models.abod import ABOD
#from pyod.models.knn import KNN
#from pyod.models.auto_encoder import AutoEncoder
#import keras
#import tensorflow as tf
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
#from sklearn.impute import KNNImputer
import scipy as sp
from sklearn.cluster import DBSCAN

In [2]:
# read file
file_name = 'baldwin_pump_data.csv'

df = pd.read_csv(file_name, parse_dates = True, index_col = 0)

In [ ]:
df.head()

In [ ]:
# null values
df_null = l.proportion_null_values(df)
df_null

In [3]:
#drop rows with all null values
df.dropna(how = 'all', axis=0, inplace = True)

In [ ]:
# check for correlation 
df.corr().to_excel('baldwin_correlation.xlsx')

In [ ]:
## Print Summary Statistics
summary_stats = l.summary_stats(df)
summary_stats

In [4]:
################ impute missing values ####################
#### KNN Impute #####
df_KNN_impute = l.KNNImpute(df,7,"uniform")

# normalise KNN imputed df
df_KNN_impute_zscale = l.zscale(df_KNN_impute)

In [ ]:
########### eliminate features using VIF ########
list_of_selected_features = df_KNN_impute_zscale.columns.tolist()

new_list_of_selected_features = l.eliminate_feature_using_VIF(df_KNN_impute_zscale,list_of_selected_features,20)

print(new_list_of_selected_features)
#save correlation matrix of selected features
#df[list_of_selected_features].corr().to_excel('selected_corr.xlsx')

In [ ]:
########### PCA + Mahalanobis Distance ################
# get optimum component for PCA
_ = optimum_component_for_PCA(df_KNN_impute_zscale)

In [6]:
## get mahalanobis Distance of vectors in data
df_PCA=l.MahalanobisDistanceOfData(df_KNN_impute_zscale,17)
df_PCA.to_excel('PCA_Cluster.xlsx')

In [ ]:
## Histogram plot of Mahalanobis distance
plt.figure(figsize=(6,5))
sns.distplot(np.square(df_x['mahala'].values), #np.square(df_x[df_x['mahala']>=600].mahala.values), 
             #bins = 50, 
             kde= False);
#plt.xticks(np.arange(0, 26000, step=1000))
#plt.yticks(np.arange(0, 120, step=5))
plt.show()

# Boxplot of Mahalanobis distance
plt.figure(figsize=(6,5))
df_x.boxplot(column = 'mahala')

In [7]:
############# KMeans ########################

clusters = range(1,100)

#Sum of squared distances of samples to their closest cluster center.
TSS = []
for i in clusters:
    kmeans = KMeans(n_clusters=i, random_state=0)
    kmeans.fit(df_KNN_impute_zscale)
    TSS.append(kmeans.inertia_)
    
plt.plot(clusters,TSS, 'bx-')
plt.show()

KeyboardInterrupt: 

In [ ]:
df_KMeans = df_transform.copy()
kmeans = KMeans(n_clusters=20, random_state=0)
cluster = kmeans.fit_predict(df_transform)

In [ ]:
df_transform['Cluster'] = cluster

In [ ]:
df_transform.to_excel('bakwas_cluster.xlsx')

In [ ]:
########## isolation forest ###############
IF = IForest(n_estimators=5000, max_samples='auto', contamination=0.1, max_features=7, bootstrap=False, n_jobs=1, behaviour='old', random_state=None, verbose=0)

In [ ]:
# predict if data is outlier
IF_OL = IF.fit_predict(df_transform)

In [ ]:
# plot histogram
plt.figure(figsize=(6,5))
plt.hist(IF_OL)
plt.yticks(np.arange(0, 10000, step=1000))
plt.show()

In [ ]:
####### DBSCAN #############
df_DBSCAN = df_transform.copy()
clustering = DBSCAN(eps=3, min_samples=2).fit(df_DBSCAN)
df_DBSCAN['Cluster'] = clustering.labels_

In [ ]:
df_DBSCAN['Cluster'].hist(bins = 10)

In [ ]:
df_DBSCAN.to_excel('DBSCAN_Cluster.xlsx')

In [ ]:
#df[df['Date'] < df.Date.iloc[0]+timedelta(days=1)].info()
# x = df[df['BFP_SUCT_HDR_PRESS________'].isnull()].index.time
## get date column
#df['Date'] = df.index.date
#df.info()